In [17]:
import numpy as np
from scipy.signal import lfilter, butter
from IPython.display import Audio
import librosa as lr

## Wah Pedal Simulation based on LPF movement and framing
I built a wah-pedal simulation program for this project. Instead of the direct function that is introduced in class, I decide to simulate the physical movement that the cutoff frequency is modulated back and forth, by doing framing on the signals. 

The basic idea of a wah pedal is to move the cutoff frequency of a low-pass filter back and forth within a certain range. As the cutoff frequency moves higher and lower periodically, the "wah" sound will be generated .

In order to create the wah pedal effect, I do the framing of the project in order to divide the original signals into small frames. I assign a low-pass filter with different cutoff frequencies (that change periodically like sinusoids) to each frame, and then combine the frames back to a new signal (just like the process of OLA). 

For the framing function, I both try to implement my own function and the function from librosa (librosa.util.frame). However, my function generate much more unwanted frequencies than the librosa function, so I use librosa.util.frame for this project.

I use a sawtooth wave with many overtones to test the signal because reading the audio file causes the Python kernel dead. It is more efficient and straightforward to test with a sawtooth wave. 

The result of the wah pedal is not perfect. Unwanted frequencies will be generated because all the frames have overlaps when they are integrated again. I put a window function for each frame in order to alleviate the unwanted effect. Still, the effect of the wah pedal can be heard in the output. 


### Functions

In [18]:
# Define the filter functions
def lowpass_filter(f0, fs):
    nyquist = 0.5 * fs
    b, a = butter(4, f0/nyquist, btype='lowpass')
    return b, a

In [19]:
# function for sawtooth wave (used for testing)
def sawtoothwave(fs,duration,f0,number_overtones,phase):
    
    # Your code here
    time_vector = np.arange(0,duration,1/fs)
    signal = np.zeros(fs*duration)
    
    for k in range(1,number_overtones+1,1): 
        signal = signal + np.sin(2*np.pi*time_vector*f0*k)/k

    signal = (-1)*signal    
        
    return time_vector, signal


In [20]:
# my own function for framing
def framing(x, frame_length, hop_length, axis=0):
    
    x = np.asarray(x) # convert input signal x to a numpy array if there isn't one
    n = x.shape[axis] # get the length of the intput signal
    n_frames = 1 + int(np.floor((n - frame_length) / hop_length)) # calculate the number of frames
    shape = list(x.shape) # define the sahpe of the output array
    shape[axis] = frame_length 
    shape.insert(axis, n_frames)  
    strides = list(x.strides) # the stride of the output array are the same as the input array
    strides = [stride * hop_length for stride in strides]
    strides.insert(axis, x.strides[axis])

    return np.lib.stride_tricks.as_strided(x, shape, strides)

### Define the variables and get the input signal

In [21]:
# Define sampling rate and frequency range
fs = 44100  # sample rate (Hz)

f_wah_min = 250  # minimum wah frequency (Hz)
f_wah_max = 1500  # maximum wah frequency (Hz)

In [22]:
# Retrieve a test signal
duration = 15  # duration of the signal (seconds)

t, x = sawtoothwave(fs, duration, 440, 20, np.pi) #get the sawtooth wave

### Framing and apply the low-pass filter

In [23]:
# Do the framing of the test signal
Nf = 256 #generate frame size
Nh = 32  #generate hop size

In [24]:
# create a 2d-array that has frame.shape(each frame's samples, # of frames)
x_frame = lr.util.frame(x, frame_length=Nf, hop_length=Nh, axis=0) #!!!!Can change to the framing function built myself here

N = x_frame.shape[0] #get the number of frames, which equals 1+int((N-Nf)/Nh)

In [25]:
wah_rate = 0.1

In [26]:
# make the cutoff frequency of the low-pass fitler to be moving and periodic
n = np.linspace(0, N/wah_rate, N) #create an array used in the for loop

sin_function = np.sin(2 * np.pi * n*1)
f_wah = f_wah_min + (f_wah_max - f_wah_min)/2 * (sin_function+1) # f_wah moves like a sinusoid periodically

In [27]:
x_wah = np.zeros_like(x_frame) #create a zero array for output

# apply a lowpass filter for each frame
for i in range (0, N): 
    b_lp, a_lp = lowpass_filter(f_wah[i], fs)
    x_wah[i, :] = lfilter(b_lp, a_lp, x_frame[i, :]) #***
    

### Integrate the frames into the output signal

In [28]:
#create the output signal
y = np.zeros(Nh*(N-1)+Nf)

#combine the frame back to the output signal
for i in range (0, N):
    y[(0+Nh*i):(0+Nh*i+Nf)] = x_wah[i, :] * np.hamming(x_wah.shape[1])

### Test

In [29]:
# Play the input signal using IPython.display.Audio
Audio(data=x, rate=fs)

In [30]:
# Play the output signal using IPython.display.Audio
Audio(data=y, rate=fs)